# Introduction

In this notebook, I will load a flatfile download of the reddit whisky reviews, parse the file, and use the URL field to query Reddit and acquire reviews.

In [1]:
import os
import uuid
import praw
import pandas as pd
import pickle as pkl

In [2]:
def url_fixer(url): 
    '''
    Function to test and fix URLs.  If the URL contains the reddit /r/ we will split there and replace with
    'https://www.reddit.com/r/'.  Otherwise, we will ensure the URL starts with http[s]:// and work from there. 
    '''
    
    if '/r/' in url:
        # There are a couple URLs where the URL is pasted twice.  Splitting on /r/ and getting the
        # last element deals with cases where there are too many values to unpack due to this double-pasting
        sub = url.split('/r/')[-1]
        return 'https://www.reddit.com/r/' + sub
    else:
        if url.startswith('http'):
            if ('reddit.com' in url) | ('redd.it' in url):
                return url
        else:
            print('url error:', url) 
            return None
        
def get_reddit_sub(url): 
    '''
    Helper Function t get reddit submissions for API - build in try/except for lambda function
    '''
    try:
        return reddit.submission(url=url)
    except:
        return None

## Load the Data File

In [3]:
whisky_data = pd.read_csv(os.getenv('DOMINO_WORKING_DIR') + '/data/raw/Reddit_Whisky_Network_Review_Archive_20190424.csv')

In [4]:
whisky_data.head()

,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review
0,12/14/2012 10:03:18,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,NaN,12/14/12
1,11/4/2018 0:50:39,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,NaN,10/30/18
2,11/17/2017 8:15:22,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,NaN,10/17/17
3,11/4/2018 0:49:15,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,$34.99,10/27/18
4,12/19/2018 0:38:02,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,NaN,12/19/2018


Let's give each row it's own unique_id we can refer back to in databases and such.

In [5]:
whisky_data['uuid'] = whisky_data.Timestamp.apply(lambda x: uuid.uuid4().hex)

In [6]:
whisky_data.to_csv(os.getenv('DOMINO_WORKING_DIR')+'/data/processed/Reddit_Whisky_Network_Review_Archive_20190424_wuuid.csv')

In [7]:
whisky_data.head()

,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review,uuid
0,12/14/2012 10:03:18,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,NaN,12/14/12,54e266b4abda4e278055f0a3254b0ef2
1,11/4/2018 0:50:39,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,NaN,10/30/18,18b5c00c30a244a49bfa15132192417c
2,11/17/2017 8:15:22,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,NaN,10/17/17,780c90af30b7447090fbc1fc1c857ecf
3,11/4/2018 0:49:15,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,$34.99,10/27/18,f86e6c62559743b4a20d85d0fb2594e8
4,12/19/2018 0:38:02,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,NaN,12/19/2018,3bc513dc90d944439df8b2f6805b7edf


## URL Check

Let's double-check that URLs are properly formatted, and at least have a /r/ substring in them that we can work from to build up proper URLS. 

In [8]:
whisky_data['url_test'] = whisky_data['Link To Reddit Review'].apply(lambda x: '/r/' in x or 'reddit.com' in x or 'redd.it' in x)

In [9]:
whisky_data.url_test.value_counts()

True     31194
False       36
Name: url_test, dtype: int64

There are 36 observations where we don't have a reddit /r/ in the URL or appear to correspond to a reddit post. Let's take a look at those.

In [10]:
whisky_data[whisky_data.url_test == False]

,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review,uuid,url_test
275,4/23/2016 17:34:00,Aberlour 12 Non Chill Filtered,bieliebielie,https://cdn2.masterofmalt.com/whiskies/p-2813/...,88,Speyside,40,04/23/16,5a62eb265a8f4570a8fd037e8306142d,False
610,5/21/2016 16:59:06,Aberlour A'bunadh Batch #53,Shawshankreddit,https://i.reddituploads.com/bdcde4da945247c9b0...,91,Speyside,£40,04/06/16,f05c2bea10634db4b7137ccaf583fd27,False
2077,5/27/2016 20:40:42,Ardbeg Uigeadail,saltpeanutss,https://c2.staticflickr.com/8/7059/6905284987_...,91,Islay,62,05/27/16,fafde542e1054e77b2f1c3bf08f36ed0,False
2604,1/13/2015 14:48:22,Arran Orkney Bere,gibbonwl,http://i.imgur.com/T20WB2F.jpg,95,Island,73,01/13/15,d57cb1f88e1a4ef59c40e1a16a799a39,False
3361,12/18/2014 12:12:47,Balvenie 12 Doublewood,CandyMan77,http://i.imgur.com/0lujJqz.jpg,86,Speyside,90,12/18/14,5d5815d1c75f4e6498a9b0170a73cf6b,False
4964,7/10/2018 13:48:51,Bernheim Original Small Batch Wheat Whiskey,FuzzyWildcat,74,74,Wheat,NaN,1/6/18,d4e867d33eba4616a2d23055120b39a5,False
7095,1/10/2015 15:57:09,Bruichladdich Scottish Barley The Classic Laddie,gildedrain,http://i.imgur.com/05FxUjj.jpg,89,Islay,57,01/10/15,b9426c88bf9e4e528047bcac40241d9b,False
7184,4/27/2016 22:03:02,Bruichladdich The Laddie Ten,lookitskeith,https://dokla.net/dailydram/wp-content/uploads...,83,Islay,NaN,04/27/16,2fc1189510984d379f7fde1b7f15fa0d,False
8837,10/18/2016 13:31:26,Cardhu 12,bpnelson7,http://imgur.com/a/JB941,85,Speyside,43,10/18/2016,a44397a19f4f47e39b87042e7b83f6c2,False
11627,8/4/2015 20:11:50,Evan Williams Single Barrel,Lasidar,http://whiskeyapostle.com/wp-content/uploads/2...,83,Bourbon,NaN,08/04/15,834eb11ad88d4c509fdfd4061c856346,False


Of the 36, most correspond to URLs outside the Reddit site, where we can't scrape with the Reddit API.  For now, we will ignore these reviews, since they only account for 0.1% of reviews.

### Fix Invalid URLs that don't start with http

In [11]:
whisky_data['parsed_url'] = whisky_data['Link To Reddit Review'].apply(url_fixer)

url error: 74
url error: 78
url error: spg2469
url error: Link To Reddit Review


In [12]:
whisky_data.head()

,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review,uuid,url_test,parsed_url
0,12/14/2012 10:03:18,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,NaN,12/14/12,54e266b4abda4e278055f0a3254b0ef2,True,https://www.reddit.com/r/Scotch/comments/14ude...
1,11/4/2018 0:50:39,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,NaN,10/30/18,18b5c00c30a244a49bfa15132192417c,True,https://www.reddit.com/r/bourbon/comments/9ssq...
2,11/17/2017 8:15:22,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,NaN,10/17/17,780c90af30b7447090fbc1fc1c857ecf,True,https://www.reddit.com/r/bourbon/comments/7445...
3,11/4/2018 0:49:15,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,$34.99,10/27/18,f86e6c62559743b4a20d85d0fb2594e8,True,https://www.reddit.com/r/bourbon/comments/9ryi...
4,12/19/2018 0:38:02,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,NaN,12/19/2018,3bc513dc90d944439df8b2f6805b7edf,True,https://www.reddit.com/r/bourbon/comments/a7jw...


## Pull URLs and Query with PRAW

In [13]:
reddit = praw.Reddit(client_id=os.getenv('reddit_clientid'),
                     client_secret=os.getenv('reddit_secret'),
                     user_agent='jbeck22')

In [14]:
whisky_queries = whisky_data[whisky_data.url_test == True][['uuid','parsed_url']]

In [15]:
whisky_queries.shape

(31194, 2)

Let's only look at the first 1000 to see how long things take.

In [16]:
test_urls = whisky_queries['parsed_url'].tolist()[1:1000]

In [17]:
posts = [reddit.submission(url=x) for x in test_urls]

In [18]:
vars(posts[0])

{'_reddit': <praw.reddit.Reddit at 0x7f8d1b178710>,
 '_fetched': False,
 '_info_params': {},
 'comment_limit': 2048,
 'comment_sort': 'best',
 'id': '9ssqyk',
 '_flair': None,
 '_mod': None,
 '_comments_by_id': {}}

So the objects here are still lazy, and we'll need to actually pull the post contents for each one. What happens if we just pull the submission titles?

In [ ]:
for i, post in enumerate(posts):
    try:
        title = post.title
    except Exception as e:
        print(e,'iteration ',i)

Some of these posts are experiencing 403 Errors and not returning anything.  It isn't clear what's happening though

In [ ]:
vars(posts[663])

In [ ]:
test_urls[663]

In [ ]:
test_urls[721]

In [ ]:
test_urls[851]

These are all coming from /r/ScotchSwap, which has been _BANNED BY REDDIT_

Either way, pulling titles results in the submission being collected, so it seems like that's all we'll need to do to save off the text

## Let's do the thing

In [ ]:
whisky_queries['submission_obj'] = whisky_queries.parsed_url.apply(get_reddit_sub)

In [ ]:
def is_fetched(submission):
    if submission is None:
        return False
    else:
        return submission._fetched
    
    
def first_comment(submission):
    try:
        submission.comment_sort = 'old'
        return list(submission.comments)[0].body
    except:
        return None
    
def selftext(submission):
    try:
        return submission.selftext
    except:
        return None

In [41]:
whisky_queries['no_sub'] = whisky_queries.submission_obj.isnull()

In [42]:
whisky_queries['fetched_flag'] = whisky_queries.submission_obj.apply(is_fetched)

In [43]:
whisky_queries.no_sub.value_counts()

False    31186
True         8
Name: no_sub, dtype: int64

In [44]:
whisky_queries.fetched_flag.value_counts()

True     31128
False       66
Name: fetched_flag, dtype: int64

In [45]:
for i, submission in enumerate(whisky_queries.submission_obj.tolist()):
    
    if submission is None or submission._fetched == True:
        continue

    try:
        title = submission.title
    except Exception as e:
        print(e, 'error on iteration ', i)
        
    if i%1000 == 0:
        print('Iteration Completed', i)
    

received 403 HTTP response error on iteration  664
received 403 HTTP response error on iteration  722
received 403 HTTP response error on iteration  852
received 403 HTTP response error on iteration  3231
received 403 HTTP response error on iteration  3235
received 403 HTTP response error on iteration  3480
received 403 HTTP response error on iteration  3527
received 403 HTTP response error on iteration  5035
received 403 HTTP response error on iteration  5456
received 403 HTTP response error on iteration  5526
received 403 HTTP response error on iteration  5633
received 403 HTTP response error on iteration  5670
received 403 HTTP response error on iteration  5724
received 403 HTTP response error on iteration  6649
received 403 HTTP response error on iteration  6665
received 403 HTTP response error on iteration  6991
received 404 HTTP response error on iteration  7100
received 403 HTTP response error on iteration  7374
received 403 HTTP response error on iteration  7752
received 403 HT

In [46]:
vars(whisky_queries['submission_obj'].tolist()[30002])

{'_reddit': <praw.reddit.Reddit at 0x7f8d1b178710>,
 '_fetched': True,
 '_info_params': {},
 'comment_limit': 2048,
 'comment_sort': 'old',
 'id': '5z5waf',
 '_flair': None,
 '_mod': None,
 '_comments_by_id': {'t1_devhk7f': Comment(id='devhk7f'),
  't1_devskk8': Comment(id='devskk8'),
  't1_devszwf': Comment(id='devszwf'),
  't1_dew0r6o': Comment(id='dew0r6o'),
  't1_dew1es7': Comment(id='dew1es7'),
  't1_dew1l59': Comment(id='dew1l59'),
  't1_dew1m0d': Comment(id='dew1m0d'),
  't1_dewb5mn': Comment(id='dewb5mn'),
  't1_dewt46w': Comment(id='dewt46w'),
  't1_dewv5vl': Comment(id='dewv5vl'),
  't1_dewveiu': Comment(id='dewveiu'),
  't1_dewwalz': Comment(id='dewwalz'),
  't1_dewwhl1': Comment(id='dewwhl1'),
  't1_dewcg7u': Comment(id='dewcg7u'),
  't1_dewdmij': Comment(id='dewdmij'),
  't1_dewdnwm': Comment(id='dewdnwm'),
  't1_dewdosg': Comment(id='dewdosg'),
  't1_devi24a': Comment(id='devi24a'),
  't1_devi3jm': Comment(id='devi3jm')},
 'approved_at_utc': None,
 'subreddit': Subreddit(

In [47]:
whisky_queries['first_comment'] = whisky_queries['submission_obj'].apply(first_comment)

In [48]:
whisky_queries['selftext'] = whisky_queries['submission_obj'].apply(selftext)

In [49]:
review_dict = {}

for i, row in whisky_queries.iterrows():
    if row.fetched_flag == True:
        review_dict[row.uuid] = {'submission': row.submission_obj,
                               'submission_id': row.submission_obj.id,
                               'first_comment': row.first_comment,
                               'selftext': row.selftext,
                               'title': row.submission_obj.title,
                               'subreddit': row.submission_obj.subreddit_name_prefixed,
                               'permalink': row.submission_obj.permalink,
                               'created_utc': row.submission_obj.created_utc
                            }

In [52]:
pkl.dump(review_dict, open('/mnt/data/processed/review_dict.pkl', 'wb'), pkl.HIGHEST_PROTOCOL)

In [53]:
test = pkl.load(open('/mnt/data/processed/review_dict.pkl', 'rb'))

In [54]:
len(test.keys())

31128

Things looked like they saved, so let's quit and move on.